### Text clustering

python -m ipykernel install --user --name Python3.5

Ref: https://stackoverflow.com/questions/39007571/running-jupyter-with-multiple-python-and-ipython-paths/39022003

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy

import os
import sys
import os.path

In [2]:
path = r"C:\Users\a.dimitrova\Desktop\Course data Thesis\clustering"

texts = []
counter = 0

# DOCUMENT LIST CONSISTS OF TEXTBLOB files. All input files need to be converted to TEXTBLOB 
# and then saved in this list in order for TF-IDF to work
doclist = []

for root, subdirs, files in os.walk(path):

    for curFile in os.listdir(root):

        filePath = os.path.join(root, curFile)

        if os.path.isdir(filePath):
            pass

        else:
            try: 
                text = ""
                curFile = open(filePath, 'r', encoding = "ISO-8859-1") #IMPORTANT ENCODING! UTF8 DOESN'T WORK
                fileExtRemoved = os.path.splitext(os.path.abspath(filePath))[0]
                
                sentences = curFile.read().split("\n")
                for sent in sentences:
                    text += sent+"\n"
                
                texts.append(text)

            finally: 
                curFile.close()

print(len(texts))

0


In [3]:
#example from online


texts = ["This first text talks about houses and dogs",
        "This is about airplanes and airlines",
        "This is about dogs and houses too, but also about trees",
        "Trees and dogs are main characters in this story",
        "This story is about batman and superman fighting each other", 
        "Nothing better than another story talking about airplanes, airlines and birds",
        "Superman defeats batman in the last round"]


# vectorization of the texts
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(texts)
# used words (axis in our multi-dimensional space)
words = vectorizer.get_feature_names()
print("words", words)


n_clusters = 4
number_of_seeds_to_try= 10
max_iter = 300
number_of_process=2 # seads are distributed
model = KMeans(n_clusters=n_clusters, max_iter=max_iter, n_init=number_of_seeds_to_try, n_jobs=number_of_process).fit(X)

labels = model.labels_
# indices of preferible words in each cluster
ordered_words = model.cluster_centers_.argsort()[:, ::-1]

print("centers:", model.cluster_centers_)
print("labels", labels)
print("intertia:", model.inertia_)

texts_per_cluster = numpy.zeros(n_clusters)
for i_cluster in range(n_clusters):
    for label in labels:
        if label==i_cluster:
            texts_per_cluster[i_cluster] +=1 

print("Top words per cluster:")
for i_cluster in range(n_clusters):
    print("Cluster:", i_cluster, "texts:", int(texts_per_cluster[i_cluster])),
    for term in ordered_words[i_cluster, :10]:
        print("\t"+words[term])

print("\n")
print("Prediction")

text_to_predict = """what we saw in the previous lecture was  how to model switch systems in a rather  general way using these things called  hybrid automaton.
today, i want to show that just because  we know how to design good linear time  invariant controllers and we know how to  draw a hybrid automaton, doesn't mean  that we can just say, [sound] we're done,  we can do everything we want to be able  to do.
in fact, what i want to do today is talk  to you about a rather famous counter  example that makes thinngs hard when  you're in the hybrid world."""
Y = vectorizer.transform([text_to_predict])
predicted_cluster = model.predict(Y)[0]
texts_per_cluster[predicted_cluster]+=1

print(text_to_predict)
print("Cluster:", predicted_cluster, "texts:", int(texts_per_cluster[predicted_cluster])),
for term in ordered_words[predicted_cluster, :10]:
    print("\t"+words[term])

words ['airlines', 'airplanes', 'batman', 'better', 'birds', 'characters', 'defeats', 'dogs', 'fighting', 'houses', 'main', 'round', 'story', 'superman', 'talking', 'talks', 'text', 'trees']
centers: [[0.         0.         0.         0.         0.         0.
  0.         0.45718777 0.         0.53486839 0.         0.
  0.         0.         0.         0.27983771 0.27983771 0.30257906]
 [0.54140529 0.54140529 0.         0.22630417 0.22630417 0.
  0.         0.         0.         0.         0.         0.
  0.16056958 0.         0.22630417 0.         0.         0.        ]
 [0.         0.         0.47031955 0.         0.         0.
  0.27204122 0.         0.29455022 0.         0.         0.27204122
  0.20899219 0.47031955 0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.52016297
  0.         0.36907117 0.         0.         0.52016297 0.
  0.36907117 0.         0.         0.         0.         0.43177993]]
labels [0 1 0 3 2 1 2]
inte